# Wavelength calibration

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### Loading crispy's dependencies

### Generate monochromatic wavelengths

In [2]:
# Suppose we create a folder in crispy/ReferenceFiles/ called wavecalR50_770_test
par.wavecalDir = par.prefix+'/wavecalR50_660/'
# construct monochromatic wavelengths, doesn't really matter which wavelengths
lamlist = np.arange(600,725,10)


In [3]:
# construct images for each wavelength
par.savePoly = True
from crispy.IFS import createWavecalFiles
createWavecalFiles(par,lamlist)

crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Writing data to ../../../crispy/SimResults/detectorFramepoly.fits
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Writing data to ../../../crispy/SimResults/detectorFramepoly.fits
crispy - INFO - Done.
crispy - INFO - Performance: 3 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_660/det_610.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming

### Loading wavelengths/filelists

In [4]:
#par.lamlist = np.loadtxt(par.wavecalDir + "lamsol.dat")[:, 0]
par.lamlist=lamlist
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))

### Run calibration. Choose parameters wisely.

In [5]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                  order=3, # Order of polynomial map of the PSFLets (between 1 and 5)
                  upsample=10, # Upsampling factor of the PSFLets, needs to be at least 5
                  nsubarr=1, # How many zones on the detector have their own PSFLets? 
                  genwavelengthsol=True, # Compute wavelength at the center of all pixels (need to do this at least once)
                  makehiresPSFlets=True, # Compute hires PSFLets (usually False)
                  makePolychrome=True,
                  apodize=False,
                  parallel=True) # Use all computer cores

crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/wavecalR50_660/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_660/det_610.fits
crispy - INFO - Initializing transformation coefficients with previous values
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../cris

In [6]:
from crispy.tools.reduction import calculateWaveList
lam_midpts,lam_endpts = calculateWaveList(par,method='lstsq')
print('Wavelengths at midpoints for lstsq (nm): ',lam_midpts)
print('Wavelengths at endpoints for lstsq (nm): ',lam_endpts)
lam_midpts,lam_endpts = calculateWaveList(par,method='optext')
print('Wavelengths at midpoints for optext (nm): ',lam_midpts)
print('Wavelengths at endpoints for optext (nm): ',lam_endpts)

crispy - INFO - Reduced cube will have 10 wavelength bins
('Wavelengths at midpoints for lstsq (nm): ', array([ 606.,  618.,  630.,  642.,  654.,  666.,  678.,  690.,  702.,  714.]))
('Wavelengths at endpoints for lstsq (nm): ', array([ 600.,  612.,  624.,  636.,  648.,  660.,  672.,  684.,  696.,
        708.,  720.]))
crispy - INFO - Reduced cube will have 19 wavelength bins
('Wavelengths at midpoints for optext (nm): ', array([ 603.15789474,  609.47368421,  615.78947368,  622.10526316,
        628.42105263,  634.73684211,  641.05263158,  647.36842105,
        653.68421053,  660.        ,  666.31578947,  672.63157895,
        678.94736842,  685.26315789,  691.57894737,  697.89473684,
        704.21052632,  710.52631579,  716.84210526]))
('Wavelengths at endpoints for optext (nm): ', array([ 600.        ,  606.31578947,  612.63157895,  618.94736842,
        625.26315789,  631.57894737,  637.89473684,  644.21052632,
        650.52631579,  656.84210526,  663.15789474,  669.47368421,
   

In [7]:
print len(par.lamlist)

13
